In [ ]:
!pip install spacy scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.3/en_core_sci_sm-0.5.3.tar.gz
!pip install pandas

  Using cached https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.3/en_core_sci_sm-0.5.3.tar.gz (14.8 MB)
  Preparing metadata (setup.py) ... done
  Using cached spacy-3.6.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (25 kB)
  Using cached thinc-8.1.12-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (15 kB)
Using cached spacy-3.6.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.6 MB)
Using cached thinc-8.1.12-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (917 kB)
  Attempting uninstall: thinc
    Found existing installation: thinc 8.2.5
    Uninstalling thinc-8.2.5:
      Successfully uninstalled thinc-8.2.5
  Attempting uninstall: spacy
    Found existing installation: spacy 3.7.5
    Uninstalling spacy-3.7.5:
      Successfully uninstalled spacy-3.7.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the follow

In [1]:
import difflib
import spacy
import re
nlp = spacy.load("en_core_sci_sm")


import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

nested_dict= {}

import pandas as pd
file_path = "/content/neurological_symptoms (1).csv"
data = pd.read_csv(file_path)
for col in data:
          if col != "Disease":
            temp = data[col].to_list()

neurological_symptoms = temp

ignore_words = ["feeling","suffering","pain","loss of","problem","anything","walking",'severe','I\'ve','\'ve','try',"problems"]

def remove_stopwords(tokens):

    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    return filtered_tokens

def extract_medical_terms(text):
    doc = nlp(text)
    medical_terms = [ent.text for ent in doc.ents]
    return medical_terms


def get_symptom_followup(sentence):

     medical_terms = extract_medical_terms(sentence)
     filtered_list = remove_stopwords(medical_terms)
     fl_list = [word for word in filtered_list if word.lower() not in ignore_words]

     matches = {}

     for word in fl_list:
       if word.lower() in neurological_symptoms:
            matches[word] = word
       else:
            closest_matches = difflib.get_close_matches(word, neurological_symptoms, n=10, cutoff=0.6)
            if closest_matches:
                matches[word] = closest_matches


     return matches


def extract_valid_duration(text):
    match = re.search(r"\b\d+\s?(hours?|days?|weeks?|months?|years?)\b", text.strip(), re.IGNORECASE)
    return match.group(0) if match else None

def ask_for_valid_duration():
    while True:
        duration = input("chatbot: Could you repeat that? Please input the duration in hours, days, months, or years.\nPatient: ")
        extracted_duration = extract_valid_duration(duration)
        if extracted_duration:
            return extracted_duration

def valid_severity(severity):
    return severity.isdigit() and 1 <= int(severity) <= 10

def ask_for_valid_severity():
    while True:
        severity = input("chatbot: Could you repeat that? How severe it feels on a scale of 1-10?\nPatient: ")
        if valid_severity(severity):
            return severity

def ask_for_valid_freq(i):
    valid_freq = ["rare", "often", "constently"]
    while True:
        freq = input(f'chatbot: How frequently do you feel {i} (Rare/Often/Constently)? \nPatient: ')
        if freq.lower() in valid_freq:
            return freq

def summary(nested_dictionary):
    summary = ''
    for symptom in nested_dictionary:
        duration = nested_dictionary[symptom].get("duration", "").strip()
        severity = nested_dictionary[symptom].get("severity", "").strip()
        frequency = nested_dictionary[symptom].get("frequency", "").strip()

        if duration:
            summary += f"You have been {frequency} experiencing {symptom} for {duration}"
        else:
            summary += f"You have been {frequency} experiencing {symptom}"

        if severity.isdigit() and 1 <= int(severity) <= 10:
            summary += f" with severity of {severity}/10.\n"
        else:
            summary += ".\n"

    return summary

def chatbot():
    symptoms = []
    print("chatbot: What is your name?")
    name = input("Patient: ")
    print(f'chatbot: Hello {name}! I’m your medical assistant chatbot. I’ll ask you a few questions to help the doctor understand your condition better.')

    while True:
        print("chatbot: Please, describe your symptoms")
        sentence = input("Patient: ")
        b = get_symptom_followup(sentence)

        for key in list(b.keys()):
          if len(b[key]) > 1 and type(b[key]) == list:
                print(f'chatbot: Which of the following symptoms are you experiencing?')
                print(f'         <<< {b[key]} >>>')
                c = input("Patient: ").split(',')
                last = c.pop()
                c.extend(last.split('and'))
                b[key] = c

        for i in list(b.values()):
            if isinstance(i, list):
                for j in i:
                    symptoms.append(j)
            else:
                symptoms.append(i)

        for i in symptoms:
          global nested_dict
          if(nested_dict.get(i) == None):
            nested_dict[i] = {"severity": "", "duration": "", "frequency": ""}
            print(f'chatbot: How long have you been experiencing {i}? (Please input duration in hours,weeks,days, months, or years.)')
            duration = input("Patient: ")
            extracted_duration = extract_valid_duration(duration)
            if not extracted_duration:
                extracted_duration = ask_for_valid_duration()
            nested_dict[i]["duration"] = extracted_duration

            print(f'chatbot: How severe it feels on a scale of 1-10?')
            severity = input("Patient: ")
            if not valid_severity(severity):
                severity = ask_for_valid_severity()
            nested_dict[i]["severity"] = severity

            frequency = ask_for_valid_freq(i)
            nested_dict[i]["frequency"] = frequency

        print(f'chatbot: Thank you. Do you have any other symptoms?')
        c = input("Patient: ")
        if 'no' in c.lower():
            print(summary(nested_dict))
            break

chatbot()


/usr/local/lib/python3.11/dist-packages/spacy/language.py:2141: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


chatbot: What is your name?
Patient: raghav
chatbot: Hello raghav! I’m your medical assistant chatbot. I’ll ask you a few questions to help the doctor understand your condition better.
chatbot: Please, describe your symptoms
Patient: i am having vision problems and  loss of reflexes and loss of muscles
chatbot: Which of the following symptoms are you experiencing?
         <<< ['coordination problems', 'memory problems'] >>>
Patient: 
chatbot: Which of the following symptoms are you experiencing?
         <<< ['reduced reflexes', 'loss of reflexes', 'restlessness'] >>>
Patient: loss of reflexes
chatbot: How long have you been experiencing ? (Please input duration in hours,weeks,days, months, or years.)


KeyboardInterrupt: Interrupted by user